# Problem Statement

Classify Devnagari letters ka and kha

csv file is given

Target :: 'character'

last columns which is name of the letter

Data explaination ::

two letters ka and kha

1024 columns which are pixels of the image

So every image is 32 * 32 size grayscale image

So one column is one pixel value

Every column has value between 0 - 255 ( grayscale image )

0 means black color and 255 means white color



In [ ]:
!unzip "/content/drive/MyDrive/DBDA/PG-DBDA March 22/Study Material/Practical Machine Learning/6 Ensemble Learning/ka_kha_classification.zip"

Archive:  /content/drive/MyDrive/DBDA/PG-DBDA March 22/Study Material/Practical Machine Learning/6 Ensemble Learning/ka_kha_classification.zip
   creating: ka_kha_classification/
  inflating: ka_kha_classification/small_image_data2.csv  


In [ ]:
import pandas as pd
df = pd.read_csv("/content/ka_kha_classification/small_image_data2.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 1026 entries, Unnamed: 0 to character
dtypes: int64(1025), object(1)
memory usage: 31.3+ MB


In [ ]:
df.columns

Index(['Unnamed: 0', 'pixel_0000', 'pixel_0001', 'pixel_0002', 'pixel_0003',
       'pixel_0004', 'pixel_0005', 'pixel_0006', 'pixel_0007', 'pixel_0008',
       ...
       'pixel_1015', 'pixel_1016', 'pixel_1017', 'pixel_1018', 'pixel_1019',
       'pixel_1020', 'pixel_1021', 'pixel_1022', 'pixel_1023', 'character'],
      dtype='object', length=1026)

In [ ]:
df.shape

(4000, 1026)

# Soplit X, Y & train test

In [ ]:
X = df.drop('character',axis=1)
Y = df['character']
X.shape,Y.shape

((4000, 1025), (4000,))

In [ ]:
# Normalization of image data
X = X / 255

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, train_size=0.7, random_state=21)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((2800, 1025), (1200, 1025), (2800,), (1200,))

# Apply XGBoost from sklearn

XGBoost outside sklearn --> https://xgboost.readthedocs.io/en/stable/python/python_intro.html

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
xgbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=7)

In [ ]:
xgbc.fit(X_train, Y_train)

GradientBoostingClassifier(learning_rate=1.0, random_state=7)

In [ ]:
Y_pred = xgbc.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))
print(accuracy_score(Y_test, Y_pred))

[[615   0]
 [  0 585]]
                  precision    recall  f1-score   support

 character_01_ka       1.00      1.00      1.00       615
character_02_kha       1.00      1.00      1.00       585

        accuracy                           1.00      1200
       macro avg       1.00      1.00      1.00      1200
    weighted avg       1.00      1.00      1.00      1200

1.0


# Fine tune the XGBoost using grid search

We want t ocheck following options

n_estimators = [ 50, 200, 500]

subsample = [0.7, 0.8, 0.9]

max_depth =[ 3, 5,7]

max_features = ['sqrt', 'log2']

In [ ]:
from sklearn.model_selection import GridSearchCV
# this is grid search CV ( cv means cross validation --> k fold cross validation)

In [ ]:
# param grid is dictionary
# key --> name of parameter to be tuned
# value --> list of values which you want to check
param_grid = {
'n_estimators' : [ 50, 200, 500],
'subsample' : [0.7, 0.8, 0.9],
'max_depth' :[ 3, 5,7],
'max_features' : ['sqrt', 'log2']
}

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gscv =GridSearchCV(estimator = GradientBoostingClassifier(), param_grid = param_grid, cv=2, verbose=2)

In [ ]:
gscv.fit(X_train,Y_train)

Fitting 2 folds for each of 54 candidates, totalling 108 fits
[CV] END max_depth=3, max_features=sqrt, n_estimators=50, subsample=0.7; total time=   0.4s
[CV] END max_depth=3, max_features=sqrt, n_estimators=50, subsample=0.7; total time=   0.6s
[CV] END max_depth=3, max_features=sqrt, n_estimators=50, subsample=0.8; total time=   0.5s
[CV] END max_depth=3, max_features=sqrt, n_estimators=50, subsample=0.8; total time=   0.4s
[CV] END max_depth=3, max_features=sqrt, n_estimators=50, subsample=0.9; total time=   0.3s
[CV] END max_depth=3, max_features=sqrt, n_estimators=50, subsample=0.9; total time=   0.3s
[CV] END max_depth=3, max_features=sqrt, n_estimators=200, subsample=0.7; total time=   0.9s
[CV] END max_depth=3, max_features=sqrt, n_estimators=200, subsample=0.7; total time=   1.1s
[CV] END max_depth=3, max_features=sqrt, n_estimators=200, subsample=0.8; total time=   1.4s
[CV] END max_depth=3, max_features=sqrt, n_estimators=200, subsample=0.8; total time=   1.4s
[CV] END max_d

GridSearchCV(cv=2, estimator=GradientBoostingClassifier(),
             param_grid={'max_depth': [3, 5, 7],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 200, 500],
                         'subsample': [0.7, 0.8, 0.9]},
             verbose=2)

In [ ]:
gscv.best_params_ # best params may differ --> because random selection of CV rows

{'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 500, 'subsample': 0.9}

In [ ]:
gscv.cv_results_['mean_test_score']

array([0.98392857, 0.99071429, 0.9825    , 0.99535714, 0.9975    ,
       0.99535714, 0.99678571, 0.99785714, 0.99821429, 0.97      ,
       0.97714286, 0.9775    , 0.98428571, 0.98928571, 0.98964286,
       0.99285714, 0.99357143, 0.995     , 0.99285714, 0.99357143,
       0.99428571, 0.99642857, 0.99714286, 0.99678571, 0.9975    ,
       0.99821429, 0.99678571, 0.98857143, 0.98642857, 0.98785714,
       0.99321429, 0.99214286, 0.99142857, 0.99535714, 0.9925    ,
       0.99428571, 0.99      , 0.99392857, 0.9925    , 0.99714286,
       0.99678571, 0.99607143, 0.99678571, 0.99642857, 0.99678571,
       0.98821429, 0.99035714, 0.98571429, 0.99285714, 0.9925    ,
       0.99142857, 0.99321429, 0.99392857, 0.99357143])

In [ ]:
gscv.cv_results_['mean_test_score'].max()

0.9982142857142857